In [1]:
import transformers
import peft
import pandas as pd
import datasets
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [2]:
def load_model(model: str):
    # load model
    m = transformers.AutoModelForCausalLM.from_pretrained(
        model,
        device_map='cuda',
        trust_remote_code=False,
        revision='main',
    )
    
    return m.to(device)

m = load_model("TheBloke/Mistral-7B-Instruct-v0.2-GPTQ")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.la

In [3]:
def load_and_prepare_dataset(model_name: str):
    df = pd.read_csv('/home/bielrossi/us-translator/data/data_openai_api_with_mask.csv')
    df = df[['text', 'is_major_principle_here', 'ethical_us']]
    instruction = 'You are a ethical requirements engineer translating requirements to ethical user stories based on 10 ethical principles of: Transparency, Non-maleficence Responsibility, Privacy, Beneficence, Freedom and autonomy, Sustainability, Dignity, Justice and Trustand the template listed below\n TEMPLATE:\n As a [persona] i want to [do something] [so that] \n\n Now following this template and the ethical principles definition, transform the requirement below into a brief description of an ethical user story, substituing [persona] for the one that requires this functionality, [do something] for what the [persona] wants to do and [so that] to the end goal of the functionality'
    df['data'] = df.apply((lambda row: f'''<s>[INST] {instruction} \n{row['text']} \n[/INST] {row['ethical_us']}</s>'''), axis=1)
    
    t = transformers.AutoTokenizer.from_pretrained(model_name, use_fast=True)
    dt = datasets.Dataset.from_pandas(df)
    dt = dt.train_test_split(test_size=0.3)

    t.pad_token = t.eos_token
    data_collator = transformers.DataCollatorForLanguageModeling(t, mlm=False)

    return dt, data_collator, t

data, data_collator, t = load_and_prepare_dataset("TheBloke/Mistral-7B-Instruct-v0.2-GPTQ")


In [4]:
def tokenize_function(examples):
    # extract text
    text = examples["data"]

    #tokenize and truncate text
    t.truncation_side = "left"
    tokenized_inputs = t(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/874 [00:00<?, ? examples/s]

Map:   0%|          | 0/219 [00:00<?, ? examples/s]

In [5]:
def train_model(model, lr, batch_size, num_epochs, tokenized_data, collator):
    model.train() # training state
    model.gradient_checkpointing_enable()
    model = peft.prepare_model_for_kbit_training(model) # turn into qlora

    # lora config
    config = peft.LoraConfig(
        r=32,
        lora_alpha=64,
        target_modules=["q_proj"],
        lora_dropout=.1,
        bias="none",
        task_type="CAUSAL_LM"
    )
    config.inference_mode = False

    model = peft.get_peft_model(model, config) # model in lora style

    training_args = transformers.TrainingArguments(
        output_dir= "checkpoints_output",
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        logging_strategy="epoch",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        fp16=False,
        optim="paged_adamw_8bit",
    )

    trainer = transformers.Trainer(
        model=model,
        train_dataset=tokenized_data["train"],
        eval_dataset=tokenized_data["test"],
        args=training_args,
        data_collator=collator
    )

    # train model
    model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
    trainer.train()

    # renable warnings
    model.config.use_cache = True

    return model

final_model = train_model(m, 2e-4, 4, 10, tokenized_data, data_collator)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,2.963500,2.913710
1,2.910700,2.913710
2,2.895100,2.913710


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_downlo

In [11]:
instruction = 'You are a ethical requirements engineer translating requirements to ethical user stories based on 10 ethical principles of: Transparency, Non-maleficence Responsibility, Privacy, Beneficence, Freedom and autonomy, Sustainability, Dignity, Justice and Trustand the template listed below\n TEMPLATE:\n As a [persona] i want to [do something] [so that] \n\n Now following this template and the ethical principles definition, transform the requirement below into a brief description of an ethical user story, substituing [persona] for the one that requires this functionality, [do something] for what the [persona] wants to do and [so that] to the end goal of the functionality'
prompt_template = lambda comment: f'''[INST] {instruction} \n{comment} \n[/INST]'''
comment = "The system shall show the meetings to the user in ascending time order by default."
prompt = prompt_template(comment)

final_model.eval()
inputs = t(prompt, return_tensors="pt")
outputs = final_model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=1024)
print(t.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] You are a ethical requirements engineer translating requirements to ethical user stories based on 10 ethical principles of: Transparency, Non-maleficence Responsibility, Privacy, Beneficence, Freedom and autonomy, Sustainability, Dignity, Justice and Trustand the template listed below
 TEMPLATE:
 As a [persona] i want to [do something] [so that] 

 Now following this template and the ethical principles definition, transform the requirement below into a brief description of an ethical user story, substituing [persona] for the one that requires this functionality, [do something] for what the [persona] wants to do and [so that] to the end goal of the functionality 
The system shall show the meetings to the user in ascending time order by default. 
[/INST] As a user who values Transparency and Beneficence, I want the system to display the meetings in ascending time order by default, so that I can easily understand the scheduling of my events and make informed decisions about my 